# Produktanalyse

## Imports

In [1]:
import pandas as pd
import pyspark
import pyspark.sql.functions as f
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

## Vorverarbeitung

In [56]:
spark = pyspark.sql.SparkSession.builder.appName("app1").getOrCreate()
sdf = spark.read.csv("data/test_data.csv", header=True, inferSchema=True)
# sdf.show()
# sdf.printSchema()


In [57]:
sdf = sdf.withColumn("category_class", pyspark.sql.functions.substring_index(sdf.category_code, '.', 1))
sdf.createOrReplaceTempView("Product_Data")

## Analysen

### SQL-Analysen

In [58]:
spark.sql("SELECT *  from Product_Data").show()

+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+--------------+
|          event_time|event_type|product_id|        category_id|       category_code|   brand| price|  user_id|        user_session|category_class|
+--------------------+----------+----------+-------------------+--------------------+--------+------+---------+--------------------+--------------+
|2019-11-01 00:00:...|      view|   1003461|2053013555631882655|electronics.smart...|  xiaomi|489.07|520088904|4d3b30da-a5e4-49d...|   electronics|
|2019-11-01 00:00:...|      view|   5000088|2053013566100866035|appliances.sewing...|  janome|293.65|530496790|8e5f4f83-366c-4f7...|    appliances|
|2019-11-01 00:00:...|      view|  17302664|2053013553853497655|                null|   creed| 28.31|561587266|755422e7-9040-477...|          null|
|2019-11-01 00:00:...|      view|   3601530|2053013563810775923|appliances.kitche...|      lg|712.87|518085591|3

In [23]:
spark.sql("SELECT COUNT(*) AS Row_Count from Product_Data").show()

+---------+
|Row_Count|
+---------+
|      217|
+---------+



#### Einzelbetrachtung

##### Product_ID

In [31]:
spark.sql("SELECT COUNT(DISTINCT(product_id)) AS Count_Unique_Product_ID from Product_Data").show()

+-----------------------+
|Count_Unique_Product_ID|
+-----------------------+
|                    188|
+-----------------------+



In [35]:
spark.sql("SELECT DISTINCT(product_id) AS Product_ID, COUNT(product_id) AS Count from Product_Data GROUP BY product_id ORDER BY Count DESC").show()

+----------+-----+
|Product_ID|Count|
+----------+-----+
|  13200605|    3|
|   5701331|    3|
|  24400437|    3|
|  12708937|    3|
|   4100297|    2|
|   1005161|    2|
|   1004856|    2|
|   1005116|    2|
|   5000088|    2|
|   1004775|    2|
|  21408240|    2|
|   1004237|    2|
|   4804295|    2|
|  38900011|    2|
|  16700826|    2|
|   1004258|    2|
|   1307115|    2|
|  17300072|    2|
|   4600603|    2|
|  29900072|    2|
+----------+-----+
only showing top 20 rows



##### Category_ID

In [36]:
spark.sql("SELECT COUNT(DISTINCT(category_id)) AS Count_Unique_Category_ID from Product_Data").show()

+------------------------+
|Count_Unique_Category_ID|
+------------------------+
|                      74|
+------------------------+



In [65]:
spark.sql("SELECT DISTINCT(category_id) AS Category_ID, COUNT(product_id) AS Count from Product_Data GROUP BY category_id ORDER BY Count DESC").show()

+-------------------+-----+
|        Category_ID|Count|
+-------------------+-----+
|2053013555631882655|   38|
|2053013558920217191|   18|
|2053013553559896355|   12|
|2053013553853497655|    9|
|2053013559901684381|    6|
|2053013556168753601|    6|
|2053013561579406073|    6|
|2053013553970938175|    5|
|2053013563911439225|    5|
|2053013554658804075|    5|
|2053013553945772349|    4|
|2053013563944993659|    4|
|2053013556311359947|    4|
|2090228413959766319|    4|
|2053013563215184719|    4|
|2053013563651392361|    4|
|2053013557024391671|    3|
|2100825583029060150|    3|
|2053013557192163841|    3|
|2060981320581906480|    2|
+-------------------+-----+
only showing top 20 rows



##### Category_Code

In [38]:
spark.sql("SELECT COUNT(DISTINCT(category_code)) AS Count_Unique_Category_Code from Product_Data").show()

+--------------------------+
|Count_Unique_Category_Code|
+--------------------------+
|                        34|
+--------------------------+



In [64]:
spark.sql("SELECT DISTINCT(category_code) AS Category_Code, COUNT(product_id) AS Count from Product_Data GROUP BY category_code ORDER BY Count DESC").show()

+--------------------+-----+
|       Category_Code|Count|
+--------------------+-----+
|                null|   87|
|electronics.smart...|   38|
|  computers.notebook|   18|
|  electronics.clocks|    8|
|furniture.kitchen...|    6|
|appliances.kitche...|    5|
|auto.accessories....|    5|
|electronics.audio...|    5|
|furniture.bedroom...|    4|
|appliances.kitche...|    4|
|construction.tool...|    4|
|electronics.audio...|    4|
|       apparel.jeans|    3|
|computers.periphe...|    2|
|furniture.living_...|    2|
|appliances.sewing...|    2|
|auto.accessories....|    2|
|electronics.camer...|    1|
|furniture.bathroo...|    1|
|construction.tool...|    1|
+--------------------+-----+
only showing top 20 rows



##### Category_class

In [60]:
spark.sql("SELECT COUNT(DISTINCT(category_class)) AS Count_Unique_Category_Class from Product_Data").show()

+---------------------------+
|Count_Unique_Category_Class|
+---------------------------+
|                          8|
+---------------------------+



In [62]:
spark.sql("SELECT DISTINCT(category_class) AS Category_Class, COUNT(product_id) AS Count from Product_Data GROUP BY category_class ORDER BY Count DESC").show()

+--------------+-----+
|Category_Class|Count|
+--------------+-----+
|          null|   87|
|   electronics|   57|
|     computers|   23|
|     furniture|   14|
|    appliances|   14|
|  construction|    8|
|          auto|    7|
|       apparel|    6|
|   accessories|    1|
+--------------+-----+



##### Brand

In [40]:
spark.sql("SELECT COUNT(DISTINCT(brand)) AS Count_Unique_Brand from Product_Data").show()

+------------------+
|Count_Unique_Brand|
+------------------+
|                85|
+------------------+



In [63]:
spark.sql("SELECT DISTINCT(brand) AS Brand, COUNT(product_id) AS Count from Product_Data GROUP BY brand ORDER BY Count DESC").show()

+---------+-----+
|    Brand|Count|
+---------+-----+
|     null|   41|
|    apple|   16|
|   xiaomi|   15|
|  samsung|   10|
|       hp|    9|
|   huawei|    6|
|   tissot|    4|
| kirkland|    4|
|      brw|    4|
|      ams|    3|
|   dewalt|    3|
|    bosch|    3|
|trussardi|    3|
|    canon|    3|
|     asus|    3|
| michelin|    3|
|     acer|    3|
|  lucente|    3|
|     acme|    2|
|     peda|    2|
+---------+-----+
only showing top 20 rows



##### Price

In [74]:
spark.sql("SELECT COUNT(DISTINCT(price)) AS Count_Unique_Price, MAX(price) AS MAX, MEAN(price) AS MEAN, MIN(price) AS MIN, AVG(price) from Product_Data").show()

+------------------+-------+-----------------+----+-----------------+
|Count_Unique_Price|    MAX|             MEAN| MIN|       avg(price)|
+------------------+-------+-----------------+----+-----------------+
|               185|2496.59|306.6343778801842|1.09|306.6343778801842|
+------------------+-------+-----------------+----+-----------------+



#### Kombibetrachtung